In [1]:
#pip install pandas
#pip install numpy
#pip install sklearn


import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
rel = pd.read_csv("answer_rel.csv")
place = pd.read_csv("place.csv")
user = pd.read_csv("user.csv")
ask = pd.read_csv('ask.csv')

In [ ]:
place_vec = rel.pivot_table(index = 'user_id', columns = 'place_id', values='id', aggfunc='count')
place_vec = place_vec.fillna(0)


from tqdm import tqdm_notebook

for i in tqdm_notebook(range(place_vec.shape[0])):
    for c in range(place_vec.shape[1]):
        if place_vec.iloc[i,c] > 0:
            place_vec.iloc[i,c] = 1

In [ ]:
drop_index_list = []
for i in place_vec.index:
    if np.sum(place_vec.loc[place_vec.index==i].values) < 8:
        drop_index_list.append(i)
        
        
place_vec = place_vec.drop(index=drop_index_list)

In [ ]:
place_taste = cosine_similarity(place_vec, place_vec) 
place_taste_sorted_ind = place_taste.argsort()[:, ::-1] 

In [159]:

def find_simi_user_final(rel_df, place_df, user_df, place_vec, sorted_ind, user_name, top_n):
    
    

    
    # 찾고자 한 유저의 id를 뽑는다
    real_user_id = user_df.loc[user_df['name'] == user_name, 'id'].values[0]
    
    
    # 유저 id를 place_vec 상의 id로 전환한다
    place_vec_reset_index = place_vec.reset_index()
    vec_user_id = place_vec_reset_index.loc[place_vec_reset_index['user_id'] == real_user_id].index.values[0]

    
    # 유저 id와 가장 유사한 장소를 고른 사람을 뽑는다
    
    similar_indexes = sorted_ind[vec_user_id, 1:(top_n + 1)]
    #similar_indexes = similar_indexes.reshape(-1)
    
    
    # 추출한 가장 비슷한 유저 id 역시 place_vec 상의 id 이므로 real id로 바꿔준다.
    real_simi_user_id_list = place_vec_reset_index.loc[place_vec_reset_index.index.isin(similar_indexes), 'user_id'].tolist()
    
    
    
    
    # 비슷한 user들 이름을 뽑아 리스트에 저장한다
    simi_user_name_list = user_df.loc[user['id'].isin(real_simi_user_id_list), 'name']
    
    
    
    d = {'simi_user_name' : simi_user_name_list,
         'simi_user_id' : real_simi_user_id_list,
         'simi_user_pick_place_id' : [place_df.loc[place_df['id'].isin(rel_df.loc[rel_df['user_id'] == u_id, 'place_id']), 'id'].tolist() for u_id in real_simi_user_id_list],
         'simi_user_pick_place_len' : [len(place_df.loc[place_df['id'].isin(rel_df.loc[rel_df['user_id'] == u_id, 'place_id']), 'id'].tolist()) for u_id in real_simi_user_id_list],
         'common_place_id' : [list(set(place.loc[place['id'].isin(rel.loc[rel['user_id'] == real_user_id, 'place_id']), 'id'].tolist()).intersection(place.loc[place['id'].isin(rel.loc[rel['user_id'] == u_id, 'place_id']), 'id'].tolist())) for u_id in real_simi_user_id_list],
         'common_place_len' : [len(list(set(place.loc[place['id'].isin(rel.loc[rel['user_id'] == real_user_id, 'place_id']), 'id'].tolist()).intersection(place.loc[place['id'].isin(rel.loc[rel['user_id'] == u_id, 'place_id']), 'id'].tolist()))) for u_id in real_simi_user_id_list]
        }
    
    
     
    result_df = pd.DataFrame(data=d, columns = ["simi_user_name", "simi_user_id", "simi_user_pick_place_id", "simi_user_pick_place_len", "common_place_id", "common_place_len"]).set_index('simi_user_id')
    result_df = result_df.sort_values(by='common_place_len', ascending=False)
    
    return result_df.drop('common_place_len', axis=1)



In [160]:
find_simi_user_final(rel, place, user, place_vec, place_taste_sorted_ind, '테드', 6)

,simi_user_name,simi_user_pick_place_id,simi_user_pick_place_len,common_place_id
simi_user_id,,,,
126,재스민,"[45, 60, 72, 76, 81, 113, 116, 188, 193, 198, ...",89,"[1325, 60, 45, 76]"
407,뺙,"[43, 44, 45, 56, 60, 74, 120, 161, 193, 194, 2...",54,"[161, 43, 60, 45]"
778,또우기,"[46, 64, 132, 181, 198, 223, 225, 271, 355, 38...",73,"[2629, 46, 959]"
56,초안,"[43, 50, 52, 53, 54, 55, 56, 78, 85, 169, 170,...",34,"[43, 78]"
40,김옥빈,"[45, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101]",11,[45]
1523,밀카쿠키,"[78, 184, 515, 680, 1746, 4367, 4370, 4371, 4372]",9,[78]
